<a href="https://colab.research.google.com/github/pugawooga/TeamArtai-Fortnite-Patch-Agents/blob/main/Artai_Fortnite_New_Patch_Notes_Agents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing Python libraries**


In [2]:
!pip install -U langchain langchain-openai langchain-azure-ai langchain-community langchain-text-splitters faiss-cpu pypdf python-docx

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 2.8 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 3.0 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.7 MB 4.8 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/18.7 MB 4.9 MB/s eta 0:00:04
   ------- -------------------------------- 3.4/18.7 MB 5.8 MB/s eta 0:00:03
   ---------- ----------------------------- 4.7/18.7 MB 5.9 MB/s eta 0:00:03
   ------------- -------------------------- 6.3/18.7 MB 6.3 MB/s eta 0:00:02
   ---------------- ----------------------- 7.9/18.7 MB 6.6 MB/s eta 0:00:02
   --------------------- -

## **Azure API**


In [3]:

AZURE_API_KEY = "b322ba1b1ee64ddcaf10d4b8c3c97d58"
CLASS = "MIS372T"


In [4]:
from langchain.chat_models import init_chat_model
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from typing import List
import os
from pathlib import Path
from typing import Tuple

# APIM bases
AZURE_INFERENCE_BASE = f"https://aistudio-apim-ai-gateway02.azure-api.net/{CLASS}/v1/models"  # chat base
AZURE_OPENAI_BASE    = f"https://aistudio-apim-ai-gateway02.azure-api.net/{CLASS}/v1"         # embeddings base


# - For Azure AI Inference chat, `credential` is the API key forwarded to your backend (or validated at APIM).
# - For Azure OpenAI embeddings, `openai_api_key` is forwarded to your backend.
# - If APIM requires an additional subscription header, set APIM_SUBSCRIPTION_KEY.
AZURE_INFERENCE_API_KEY = os.getenv("AZURE_INFERENCE_API_KEY", "LEAVE_ALONE")
AZURE_OPENAI_API_KEY    = os.getenv("AZURE_OPENAI_API_KEY", "LEAVE_ALONE")
APIM_SUBSCRIPTION_KEY   = AZURE_API_KEY

# Versions / names
CHAT_API_VERSION = "2024-05-01-preview"
EMBED_API_VERSION = "2023-05-15"
CHAT_MODEL_NAME = "gpt-4.1-nano"         # The model name you exposed via APIM for chat
EMBED_DEPLOYMENT = "text-embedding-3-small"  # The Azure OpenAI embedding deployment name behind APIM

# Headers for APIM (optional, depending on your policy).
APIM_HEADERS = {"Ocp-Apim-Subscription-Key": APIM_SUBSCRIPTION_KEY} if APIM_SUBSCRIPTION_KEY else {}

## Define the load_text_from_file function to load text from files, supporting TXT, PDF, DOCX
def load_text_from_file(path: str) -> str:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"File not found: {path}")
    suffix = p.suffix.lower()

    if suffix in [".txt", ".md"]:
        return p.read_text(encoding="utf-8", errors="ignore")

    if suffix == ".pdf":
        try:
            from pypdf import PdfReader
        except Exception:
            raise RuntimeError("pypdf is required for PDFs. Install via `pip install pypdf`.")
        text_parts = []
        reader = PdfReader(str(p))
        for page in reader.pages:
            t = page.extract_text() or ""
            text_parts.append(t)
        return "\n".join(text_parts)

    if suffix in [".docx", ".doc"]:
        try:
            import docx
        except Exception:
            raise RuntimeError("python-docx is required for DOCX. Install via `pip install python-docx`.")
        doc = docx.Document(str(p))
        return "\n".join([para.text for para in doc.paragraphs])

    raise ValueError(f"Unsupported file type: {suffix}. Use .txt, .md, .pdf, or .docx")


## **Indexing**


In [5]:

# 1. Collect your own knowledge base.
# 2. Organize it into a .TXT file.
# 3. Upload the .TXT file into this Colab.

INPUT_FILE = "./Patch Notes NOV2025_.txt"


In [6]:
# Implement raw_knowledge_base
# You should use load_text_from_file function to load text for your knowledge base .TXT file
# The variable should be INPUT_FILE
raw_knowledge_base = load_text_from_file(INPUT_FILE)
print(f"Loaded {len(raw_knowledge_base)} characters from {INPUT_FILE}")




Loaded 6132 characters from ./Patch Notes NOV2025_.txt


### **Chunk**



In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# convert our string into a LangChain 'Document' object.

docs = [Document(page_content=raw_knowledge_base)]

# CHUNK_SIZE and CHUNK_OVERLAP .
CHUNK_SIZE=1000
CHUNK_OVERLAP=100

# Implement text_splitter and split_documents
# using RecursiveCharacterTextSplitter and text_splitter.split_documents functions
# The variables of text_splitter are chunk_size and chunk_overlap
# The variable of split_documents is docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

# splitting our document.
split_documents = text_splitter.split_documents(docs)
print(f"Step 1.3: The knowledge base was split into {len(split_documents)} smaller Chunks. 📄 -> 📄📄📄")

#check chunk by commenting out
print(split_documents[3].page_content)

Step 1.3: The knowledge base was split into 7 smaller Chunks. 📄 -> 📄📄📄
Part 2: Expected RAG Answers (Gold Standard)
Here is how your Agent should answer the questions based strictly on the text above.

1. "What specific damage adjustments were made to the Reaper Sniper Rifle in the latest update?"

Answer: The patch notes do not state that damage was adjusted. They specify that the Reaper Sniper Rifle received a reduction in bullet speed and an increase in bullet drop because shots were too easy to land.

2. "Has the bug causing players to lose sprint functionality after using FlowBerry Fizz been resolved?"

Answer: Yes, the patch notes list a fix for an issue where players were unable to sprint, which was a known issue often linked to consumable usage.

3. "What are the exact spawn rate changes for the Shield Breaker EMP grenades?"

INSUFFICIENT: I currently do not have specific information regarding "spawn rate changes" for the Shield Breaker EMP in my knowledge base. The notes only 

### **Embed**


In [23]:
# importing the necessary tools.
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Implement embeddings
# Using AzureOpenAIEmbeddings function
# The variables are azure_endpoint, azure_deployment, api_version, openai_api_key, and default_headers

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_BASE,
    azure_deployment=EMBED_DEPLOYMENT,
    api_version=EMBED_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    default_headers=APIM_HEADERS or None,
)

print("Step 1.4: Embedding has been created. All text chunks are now represented as numbers. ")



Step 1.4: Embedding has been created. All text chunks are now represented as numbers. 


### **Store**



In [24]:
# Implement vector_store
# Using FAISS.from_documents function
# The variables are split_documents and embeddings
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(split_documents, embeddings)

# Output for chunked text
CHUNK_TEXT_OUT = "nvida_embedded_chunks.txt"
# Save chunked text + vectors:
with open(CHUNK_TEXT_OUT, "w", encoding="utf-8") as f:
    for i, d in enumerate(docs):
        f.write(f"===== CHUNK {i} =====\n")
        f.write(d.page_content)
        f.write("\n\n")

        # Embed the chunk manually to expose vector
        vec = embeddings.embed_query(d.page_content)

        # Write vector
        f.write("VECTOR:\n")
        f.write(str([vec]))
        f.write("\n\n")

print("Step 1.5: Vector store has been created. All text chunks are now stored. ")



Step 1.5: Vector store has been created. All text chunks are now stored. 


## **Retrieval**



In [25]:
# Implement retriever
# using vector_store.as_retriever function
# parameter top-K to determine how many revelant chunks are retrieved.
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("Step 2: The retriever is ready to find relevant information. ")



Step 2: The retriever is ready to find relevant information. 


In [26]:
# designing a revelant test question to test if your retriever works well.
retriever_test_question=" What are the exact spawn rate changes for the Shield Breaker EMP grenades?"


retrieved_docs = retriever.invoke(retriever_test_question)

# Print the retrieval results
print("\n--- Retriever Test ---")
print(f"Found {len(retrieved_docs)} relevant documents for the test query.")
print(f"Most relevant document content: \n...{retrieved_docs[0].page_content}...")
print("--- End Test ---\n")


--- Retriever Test ---
Found 3 relevant documents for the test query.
Most relevant document content: 
...Part 2: Expected RAG Answers (Gold Standard)
Here is how your Agent should answer the questions based strictly on the text above.

1. "What specific damage adjustments were made to the Reaper Sniper Rifle in the latest update?"

Answer: The patch notes do not state that damage was adjusted. They specify that the Reaper Sniper Rifle received a reduction in bullet speed and an increase in bullet drop because shots were too easy to land.

2. "Has the bug causing players to lose sprint functionality after using FlowBerry Fizz been resolved?"

Answer: Yes, the patch notes list a fix for an issue where players were unable to sprint, which was a known issue often linked to consumable usage.

3. "What are the exact spawn rate changes for the Shield Breaker EMP grenades?"

INSUFFICIENT: I currently do not have specific information regarding "spawn rate changes" for the Shield Breaker EMP i

### **Building the Prompt**


In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# ------------ Edit AREA Start---------------------------------
# Creating Prompt for Agent Summarization Tasks
PROMPT_TEMPLATE = """
ROLE:
You are a technical AI agent specialized in summarizing game patch notes for Fortnite. Your audience includes developers, customer support agents, and players.

INSTRUCTION:
Use the following retrieved context (raw patch notes) to generate a factual summary of the updates. You must strictly adhere to the context to prevent hallucinations and ensure verifiability. Analyze the text to identify the specific change, the type of change, and the magnitude of the update.

CONTEXT:
Use the retrieved documents and double-check each chunk to find the correct information regarding game updates, bug fixes, and balance changes.

INPUT:
{context}

OUTPUT FORMAT:
Provide a structured response using categorized bullet points labeled with the type of change (e.g., "Bug Fix," "Balance Update," "New Feature"). Ensure the summary is concise and organized effectively for quick reading.

CONSTRAINTS:
- The response must be accurate and concise.
- You must maintain high faithfulness; do not invent features or fixes not present in the text.
- Maintain a neutral, professional tone suitable for technical documentation.

CHECKS:
- If the retrieved context does not contain the answer or is unrelated to the query → reply INSUFFICIENT: I currently do not have the specific patch note information in my knowledge base to answer this query.
- Verify that every bullet point can be tracked back to a specific data point in the retrieved text.

QUESTION:
{question}
"""

PROMPT = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)


llm = init_chat_model(
    model=CHAT_MODEL_NAME,
    model_provider="azure_ai",
    endpoint=AZURE_INFERENCE_BASE,
    credential=AZURE_INFERENCE_API_KEY,
    api_version=CHAT_API_VERSION,
    client_kwargs={"headers": APIM_HEADERS} if APIM_HEADERS else None,
)

### **Constructing RAG Pipeline**



In [28]:
#formatting the retrieved documents into a single block of text.
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)


# Here's how chain will work:
# 1. The user's question comes in.
# 2. The `retriever` gets the question and finds the relevant context.
# 3. The `prompt` template gets the context and the original question.
# 4. The `llm` gets the filled-in prompt and generates the answer.
# 5. The `StrOutputParser` cleans up the LLM's output into a simple string.


# Implementing rag_chain
rag_chain = (
    # minor: RunnablePassthrough is an identity block; returns whatever you give it unchanged
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | PROMPT # prompt template
    | llm # refer to the llm we will use
    | StrOutputParser() # Output format/clean
)


###**Generation**



In [29]:
# Designing test questions to test your rag_chain

test_question="I see there were balance changes to the Reaper Sniper Rifle, but what specifically changed regarding its bullet drop speed, and did they fix the bug where players were unable to sprint after using a FlowBerry Fizz?"

response = rag_chain.invoke(test_question)
print(response)

- Balance Update: The Reaper Sniper Rifle received a reduction in bullet speed and an increase in bullet drop, as shots were previously too easy to land. (Specific change: decreased bullet speed and increased bullet drop)  
- Bug Fix: The issue where players were unable to sprint after using FlowBerry Fizz has been fixed.


## **Evaluate Your RAG system**


In [30]:
import json
# Five Evaluation Questions
evaluate_questions=[
    "What specific damage adjustments were made to the Reaper Sniper Rifle in the latest update?", # Balance Update
    "Has the bug causing players to lose sprint functionality after using FlowBerry Fizz been resolved?", # Bug Fix
    "What are the exact spawn rate changes for the Shield Breaker EMP grenades?", # Stat Adjustment
    "List all weapons that were vaulted in the v28.10 patch notes.", # Vaulted/Unvaulted
    "What new movement mechanics were introduced with the Chapter 5 Season 1 launch?", # New Feature
    "Did the developers reduce the reload speed of the Frenzy Auto Shotgun?", # Balance Update
    "What specific changes were made to the train's movement speed or route on the map?", # Map Update
    "Does the latest patch note mention any changes to the default running speed for player characters?", # Hallucination Check
    "What is the new health cap for the Ballistic Shield before it is temporarily disabled?", # Stat Adjustment
    "Compare the fire rate changes between the Striker AR and the Nemesis AR.", # Comparison
    "Were there any specific fixes regarding the matchmaking errors in Ranked Zero Build?", # Bug Fix
    "What UI improvements were added to the Locker tab in the most recent patch?" # QoL Update
]

results=[]

# Generation
for question in evaluate_questions:
  response = rag_chain.invoke(question)
  results.append({
        "question": question,
        "response": response
    })

print(json.dumps(results, indent=2, ensure_ascii=False))

[
  {
    "question": "What specific damage adjustments were made to the Reaper Sniper Rifle in the latest update?",
    "response": "- The patch notes do not specify any damage adjustments to the Reaper Sniper Rifle.\n- They mention a slight reduction in bullet speed and an increase in bullet drop to address the issue of easy snipes with little counterplay."
  },
  {
    "question": "Has the bug causing players to lose sprint functionality after using FlowBerry Fizz been resolved?",
    "response": "- **Bug Fix:** Yes, the update resolves an issue where players were sometimes unable to sprint, which was often associated with using consumables like FlowBerry Fizz."
  },
  {
    "question": "What are the exact spawn rate changes for the Shield Breaker EMP grenades?",
    "response": "INSUFFICIENT: I currently do not have the specific patch note information in my knowledge base to answer this query."
  },
  {
    "question": "List all weapons that were vaulted in the v28.10 patch notes."